In [1]:
import requests
import time
import numpy as np

In [2]:
gm = np.load("puuids.npy")
m = np.load("masters.npy") #jusqu'a 10000
puuids = list(gm) + list(m)

In [3]:
key = 'RGAPI-0e2f42e5-53ed-4ca7-9551-c5651916ca25'

In [4]:
def get_puuid(puuids,list_player,key):
    c=1
    for p in list_player:
        sum_name = p['summonerName']
        try:
            req = requests.get('https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{}?api_key={}'.format(sum_name,key))
            r = req.json()
            puuid = r['puuid']
            puuids.append(puuid)
        except Exception:
            pass
        time.sleep(0.9)
        print(len(puuids))
        #print(puuid)
        c+=1

In [5]:
def get_matches_id(m_ids,puuids,key):
    c=1
    for p in puuids:
        req = requests.get('https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{}/ids?start=0&count=99&api_key={}'.format(p,key))
        r = req.json()
        try:
            for m in r:
                m_ids.append(m)
        except:
            print(r)
        time.sleep(0.9)
        print(len(m_ids))
        c+=1

In [6]:
GM_Match = np.load("grand_matches_id.npy")

M_Match = np.load("master_matches40000.npy") # Jusqu'a 400000 (avec doublon)

In [7]:
m_ids = list(GM_Match) + list(M_Match)
m_ids = list(set(m_ids))

In [8]:
len(m_ids)

319241

In [31]:
def add_match(match_id,api_key):
    Match = []
    req = 'https://europe.api.riotgames.com/lol/match/v5/matches/{}?api_key={}'.format(match_id,api_key)
    time.sleep(1.2)
    r = requests.get(req)
    match_info = r.json()
    if match_info['info']['queueId']==420:
        Blue,Red = [],[]
        players = match_info['info']['participants']
        Winner = "Blue" if players[0]['win'] else "Red"
        for i in range(len(players)):
            if i<5:
                Blue.append(players[i]['individualPosition'])
                Blue.append(players[i]['championName'])
            else:
                Red.append(players[i]['individualPosition'])
                Red.append(players[i]['championName'])
        Match = [Blue,Red,Winner]
    return Match

In [32]:
def match(m_ids,key):
    Matchs = []
    for m in m_ids:
        ma = add_match(m,key)
        if ma!=[]:
            Matchs.append(ma)
        if len(Matchs)==1000:
            name = "data\\Matchs{}.npy".format(m_ids.index(m))
            Matchs = np.array(Matchs,dtype=list)
            np.save(name,Matchs)
            print("saved {}".format(name))
            Matchs = []

In [ ]:
match(m_ids,key)